In [54]:
import pickle

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

from pt_runner.cnn import CheckpointHandler, DataHandlerPT, EarlyStopper

In [55]:
# New run
NEW_RUN = True
DT_REF = None

# Resuming
# NEW_RUN = False
# DT_REF = "2025-05-28_10-43"


In [56]:
RANDOM_STATE = 0

In [57]:
with open("mnist_small.pickle", "rb") as file:
    data = pickle.load(file)

In [58]:
_X = data["_X"].astype(np.float64)
_Y = data["_Y"].astype(np.int32)
print(_X.shape)
print(_X.dtype)
print(_Y.shape)
print(_Y.dtype)

(2000, 1, 28, 28)
float64
(2000, 1)
int32


In [59]:
data_handler = DataHandlerPT(_X=_X, _Y=_Y)

In [60]:
import torch.nn as nn


class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        # First convolutional layer: input channels=3 (e.g., RGB), output channels=16, kernel size=3
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        
        self.max_pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))
        # Fully connected layer
        self.fc1 = nn.Linear(32 * 4 * 4, num_classes)  # Adjust input size depending on image size!

    def forward(self, X):
        # 1st Conv + Activation + Pooling
        X = self.conv1(X)
        X = self.relu(X)
        X = self.max_pool(X)
        # 2nd Conv + Activation + Pooling
        X = self.conv2(X)
        X = self.relu(X)
        X = self.max_pool(X)

        X = self.adaptive_pool(X)
        # Flatten the output for the fully connected layer
        X = X.view(X.shape[0], -1)
        # Fully connected output
        X = self.fc1(X)
        return X

model = SimpleCNN(num_classes=10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=5)
loss_fn = nn.CrossEntropyLoss()


# Test
X = torch.randn(10,1, 28,28)
model(X)

tensor([[ 0.4285,  0.5164, -0.1503, -0.3107, -0.2864, -0.1454, -0.2039, -0.3829,
          0.4990,  0.0946],
        [ 0.4150,  0.5385, -0.1328, -0.2854, -0.2999, -0.1560, -0.2194, -0.4265,
          0.5209,  0.0918],
        [ 0.3686,  0.5573, -0.1741, -0.2945, -0.2798, -0.1405, -0.2148, -0.4065,
          0.4198,  0.1498],
        [ 0.4004,  0.5316, -0.1958, -0.3578, -0.3306, -0.1335, -0.2337, -0.3938,
          0.4864,  0.0839],
        [ 0.4165,  0.5287, -0.1768, -0.3492, -0.3555, -0.1698, -0.2517, -0.4273,
          0.4986,  0.0974],
        [ 0.4087,  0.4957, -0.1275, -0.3625, -0.3103, -0.1728, -0.2275, -0.3894,
          0.4445,  0.0719],
        [ 0.3778,  0.5421, -0.1936, -0.3965, -0.3241, -0.1704, -0.2628, -0.3800,
          0.5135,  0.0715],
        [ 0.3977,  0.5377, -0.1756, -0.3325, -0.3044, -0.1376, -0.2335, -0.4195,
          0.4738,  0.1065],
        [ 0.3839,  0.5326, -0.1739, -0.2993, -0.3323, -0.1688, -0.2065, -0.4378,
          0.4514,  0.0785],
        [ 0.3627,  

In [61]:
from torchinfo import summary

input_size = (100, 1, 32, 32) # (batch_size, channels, height, width)
summary(model, input_size=input_size)

Layer (type:depth-idx)                   Output Shape              Param #
SimpleCNN                                [100, 10]                 --
├─Conv2d: 1-1                            [100, 16, 32, 32]         160
├─ReLU: 1-2                              [100, 16, 32, 32]         --
├─MaxPool2d: 1-3                         [100, 16, 16, 16]         --
├─Conv2d: 1-4                            [100, 32, 16, 16]         4,640
├─ReLU: 1-5                              [100, 32, 16, 16]         --
├─MaxPool2d: 1-6                         [100, 32, 8, 8]           --
├─AdaptiveAvgPool2d: 1-7                 [100, 32, 4, 4]           --
├─Linear: 1-8                            [100, 10]                 5,130
Total params: 9,930
Trainable params: 9,930
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 135.68
Input size (MB): 0.41
Forward/backward pass size (MB): 19.67
Params size (MB): 0.04
Estimated Total Size (MB): 20.12

In [62]:
n_epochs = 1000  # number of epochs to run
batch_size = 10  # size of each batch
validation_interval = 10  # Evaluate every 100 epochs
log_name = "M1"
random_state = 0  # Split data

# Save/load
cph = CheckpointHandler()
cph.make_dir("./checkpoints")
if NEW_RUN:
    dt = cph.get_dt()
    log_dir = f"runs/{dt}"
    save_path = f"./checkpoints/{dt}.pth"
    epoch_start = 0
else:
    log_dir = f"runs/{DT_REF}"
    load_path = f"./checkpoints/{DT_REF}.pth"
    save_path = load_path
    model, optimizer, epoch, val_loss = cph.load(
        load_path=load_path, model=model, optimizer=optimizer
    )
    epoch_start = epoch
    print(f"Resuming from epoch: {epoch}")

epoch_end = epoch_start + n_epochs

# Initialize Components
early_stopper = EarlyStopper(patience=10)
writer = SummaryWriter(log_dir=log_dir, purge_step=epoch_start)

# Data
data_handler.split_and_scale(test_size=0.2, val_size=0.1, random_state=RANDOM_STATE)
ds_train = data_handler.get_train()
ds_test = data_handler.get_test()
ds_val = data_handler.get_val()
loader_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(ds_val, batch_size=batch_size, shuffle=False)

# Main loop
for epoch in tqdm(
    range(epoch_start, epoch_end), initial=epoch_start, desc="Epoch", total=n_epochs
):
    # Training Phase
    model.train()
    epoch_train_loss = 0.0

    for X_batch, Y_batch in loader_train:
        optimizer.zero_grad()
        Y_pred = model(X_batch)
        loss = loss_fn(Y_pred, Y_batch.view(-1))
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        # Update weights
        optimizer.step()
        # Multiplies the average loss per sample by the number of
        # samples in the batch to get the total loss for this batch.
        epoch_train_loss += loss.item() * X_batch.size(0)

    avg_train_loss = epoch_train_loss / len(loader_train.dataset)

    # Validation Phase
    if epoch % validation_interval == 0 or epoch == epoch_start:
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_val, Y_val in loader_val:
                Y_pred = model(X_val)
                val_loss += loss_fn(Y_pred, Y_val.view(-1)).item() * X_val.size(0)

        avg_val_loss = val_loss / len(loader_val.dataset)
        scheduler.step(avg_val_loss)

        # Early Stopping and Checkpoint
        es = early_stopper(avg_val_loss)
        if es["best_loss"]:
            cph.save(
                save_path=save_path,
                model=model,
                optimizer=optimizer,
                val_loss=avg_val_loss,
                epoch=epoch,
            )
            print("Save model @ epoch:", epoch)
        if es["early_stop"]:
            print("Stopped at epoch:", epoch)
            break

        writer.add_scalars(
            log_name, {"train_loss": avg_train_loss, "val_loss": avg_val_loss}, epoch
        )


Epoch:   0%|          | 0/1000 [00:00<?, ?it/s]

Save model @ epoch: 0
Save model @ epoch: 10
Stopped at epoch: 110


In [ ]:
ds_test[0]